In [ ]:
import tensorflow
print(tensorflow.__version__)

In [ ]:
import tensorflow as tf
help(tf.Variable)

In [ ]:
x=tf.Variable(3,name="x")
y=tf.Variable(4,name="y")
f=x**2*y+x*y+y+2
sess=tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result=sess.run(f)
print(result)
sess.close()

In [ ]:
help(x.initializer)

In [ ]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result1=f.eval()
    print(result1)
#tf.get_default_session().run()

In [ ]:
init=tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result=f.eval()
    print(result)

In [ ]:
x1=tf.Variable(1)
x1.graph is tf.get_default_graph()
graph=tf.Graph()
with graph.as_default():
    x2=tf.Variable(2)
x2.graph is graph
x2.graph is tf.get_default_graph()
#tf.get_default_session().run()

In [ ]:
w=tf.constant(3)
x=w+2
y=x*3
z=y**w
with tf.Session() as ess:
    print(y.eval())
    print(z.eval())

In [ ]:
with tf.Session() as sess:
    y_val,z_val=sess.run([y,z])
    print(y_val)
    print(z_val)
print(tf.get_default_session())

In [ ]:
import numpy as np
import sklearn.datasets
from sklearn.datasets import fetch_california_housing
#dir(sklearn.datasets)
housing=fetch_california_housing()
m,n=housing.data.shape
housing_data_plus_bias=np.c_[np.ones((m,1)),housing.data]
X=tf.constant(housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
XT=tf.transpose(X)
theta=tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)
with tf.Session() as sess:
    theta_value=theta.eval()
    print(theta_value)

In [ ]:
#m,n
#help(np.ones)
housing_data_plus_bias.shape,housing.data.shape,theta_value.shape
#housing_data_plus_bias[:,0]

In [ ]:
dir(housing)
housing.feature_names

In [15]:
from sklearn.preprocessing import MinMaxScaler
mmx=MinMaxScaler()
scaled_data=mmx.fit_transform(housing.data)
scaled_housing_data_plus_bias=np.c_[np.ones((m,1)),scaled_data]

In [ ]:
n_epochs=10000
learning_rate=0.2
X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name='theta')
y_pred=tf.matmul(X,theta,name='predictions')
error=y_pred-y
mse=tf.reduce_mean(tf.square(error),name='mse')
gradients=2/m*tf.matmul(tf.transpose(X),error)
training_op=tf.assign(theta,theta-learning_rate*gradients)
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
            print("Epoch",epoch,"MSE=",mse.eval())
        sess.run(training_op)
    best_theta=theta.eval()
    print(best_theta)

In [ ]:
best_theta,housing.feature_names

In [ ]:
optimizer= tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op=optimizer.minimize(mse)
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
            print("Epoch",epoch,"MSE=",mse.eval())
        sess.run(training_op)
    best_theta=theta.eval()
    #print("Epoch",epoch,"MSE=",mse.eval())
    print(best_theta)

In [ ]:
import tensorflow as tf
#import tensorflow.metrics as tm
n_epochs=10000
learning_rate=0.6
X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name='theta')
y_pred=tf.matmul(X,theta,name='predictions')
#mse1=tf.metrics.mean_squared_error(y,y_pred)[0]
mse=tf.Variable(tf.metrics.mean_squared_error(y,y_pred)[0],name='mse')
total_error = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
unexplained_error = tf.reduce_sum(tf.square(tf.subtract(y, y_pred)))
R_squared = tf.subtract(1.0, tf.div(unexplained_error, total_error),name='R_squared')
#optimizer= tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.1)
training_op=optimizer.minimize(-R_squared)
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch%100==0:
            print("Epoch",epoch,"R_2=",R_squared.eval())
        sess.run(training_op)
    best_theta=theta.eval()
    #print("Epoch",epoch,"R_2=",R_squared.eval())
    print(best_theta)

In [ ]:
dir(tf.metrics)
#help(optimizer.variables)

In [ ]:
import tensorflow as tf
A=tf.placeholder(tf.float32,shape=(None,3))
B=A+5
with tf.Session() as sess:
    B_val_1=B.eval(feed_dict={A:[[1,2,3]]})
    B_val_2=B.eval(feed_dict={A:[[4,5,6],[5,6,7]]})
print(B_val_1)
print(B_val_2)

In [ ]:
import numpy as np
import sklearn.datasets
from sklearn.datasets import fetch_california_housing
#dir(sklearn.datasets)
housing=fetch_california_housing()
m,n=housing.data.shape
X=tf.placeholder(tf.float32,shape=(None,n+1),name="X")
y=tf.placeholder(tf.float32,shape=(None,1),name="y")

In [ ]:
batch_size=100
n_epochs=10000
learning_rate=0.6
n_batches=int(np.ceil(m/batch_size))
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch,y_batch=fetch_batch(epoch,batch_index,batch_size)
            sess.run(training_op,feed_dict={X:X_batch,y:y_batch})            
    best_theta=theta.eval()

In [13]:
import pandas as pd
dat=pd.read_csv("train_data.csv")
X=dat.iloc[:,:-1]
y=dat.iloc[:,-1]
X.shape,y.shape

((614, 12), (614,))

# Saving the tensorflow model

In [23]:
import tensorflow as tf
#import tensorflow.metrics as tm
n_epochs=10000
learning_rate=0.6
X=tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32,name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")
theta=tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name='theta')
y_pred=tf.matmul(X,theta,name='predictions')
#mse1=tf.metrics.mean_squared_error(y,y_pred)[0]
mse=tf.Variable(tf.metrics.mean_squared_error(y,y_pred)[0],name='mse')
total_error = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
unexplained_error = tf.reduce_sum(tf.square(tf.subtract(y, y_pred)))
R_squared = tf.subtract(1.0, tf.div(unexplained_error, total_error),name='R_squared')
#optimizer= tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer=tf.train.MomentumOptimizer(learning_rate=learning_rate,momentum=0.1)
training_op=optimizer.minimize(-R_squared)
init=tf.global_variables_initializer()
saver=tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        #if epoch%100==0:
            #print("Epoch",epoch,"R_2=",R_squared.eval())
        sess.run(training_op)
    best_theta=theta.eval()
    #print("Epoch",epoch,"R_2=",R_squared.eval())
    print(best_theta)
    saver_path=saver.save(sess,"../Documents/my_model_final.ckpt")

[[ 4.0620403 ]
 [ 5.552959  ]
 [ 0.48999682]
 [-1.0305543 ]
 [ 5.3627887 ]
 [-0.12867244]
 [-2.744023  ]
 [-4.287504  ]
 [-4.6597633 ]]
